In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import os
import urllib.request
import urllib.parse
import requests
import pyfits
import numpy
import scipy.signal

class lamost:
    email=None
    token=None
    dataset=None
    version=None
    __isdev=False

    def __init__(self, isdev=False, dataset=5):
        self.__isdev=isdev
        self.dataset=dataset

    def __getDataset(self):
        prefix='dr5'
        if self.dataset is not None:
            prefix = 'dr%d'%self.dataset
        if self.__isdev: return 'l'+prefix
        else: return prefix

    def __getVersion(self):
        if self.version is not None:
            return '/v%d'%self.version
        return ''

    __config=None
    __config_file_path=os.path.expanduser('~')+'/pylamost.ini'
    
    def __getConfig(self, reload=False):
        if not os.path.exists(self.__config_file_path): return None
        
        if not reload and None!=self.__config: return self.__config
        
        with open(self.__config_file_path) as fh:
            self.__config={}
            for line in fh:
                if line.startswith('#'):continue
                k,v=line.split("=")
                self.__config[k.strip()]=v.strip()
        return self.__config

    def __detectToken(self):
        if self.token is not None: return True
        cf = self.__getConfig()
        if cf is None or 'token' not in cf.keys(): 
            print('please set your token')
            return False
        self.token=cf['token']
        return True

    def download(self, url, savedir='./'):
        response = urllib.request.urlopen(url)
        data = response.read()
        savefile=savedir+'/'+response.getheader("Content-disposition").split('=')[1]
        with open(savefile, 'wb') as fh:
            fh.write(data)

    def getUrl(self, url, params=None):
        if params is None:
            response = urllib.request.urlopen(url)
        else:
            response = urllib.request.urlopen(url, urllib.parse.urlencode(params).encode('utf-8'))
        chrset = response.headers.get_content_charset()
        if chrset is None: chrset='utf-8'
        data = response.read().decode(chrset)
        return data

    def downloadCatalog(self, catname, savedir='./'):
        caturl='http://{0}.lamost.org{1}/catdl?name={2}&token={3}'.format(self.__getDataset(), self.__getVersion(), catname, self.token)
        self.download(url, savedir)

    def downloadFits(self, obsid, savedir='./'):
        if not self.__detectToken(): return
        fitsurl='http://{0}.lamost.org{1}/spectrum/fits/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        self.download(fitsurl, savedir)

    def downloadPng(self, obsid, savedir='./'):
        if not self.__detectToken(): return
        pngurl='http://{0}.lamost.org{1}/spectrum/png/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        self.download(pngurl, savedir)

    def getFitsCsv(self, obsid):
        if not self.__detectToken(): return None
        url='http://{0}.lamost.org{1}/spectrum/fits2csv/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        return self.getUrl(url)

    def getInfo(self, obsid):
        if not self.__detectToken(): return None
        #url='http://{0}.lamost.org{1}/spectrum/info/{2}?token={3}'.format(self.__getDataset(), self.__getVersion(), obsid, self.token)
        #return self.getUrl(url, params)
        url='http://{0}.lamost.org{1}/spectrum/info/{2}'.format(self.__getDataset(), self.__getVersion(), obsid)
        return self.getUrl(url, {'token':self.token})

    def conesearch(self, ra, dec, radius):
        if not self.__detectToken(): return
        conesearchurl='http://{0}.lamost.org{1}/voservice/conesearch?ra={2}&dec={3}&sr={4}&token={5}'.format(self.__getDataset(), self.__getVersion(), ra, dec, radius, self.token)
        return self.getUrl(conesearchurl)

    def ssap(self, ra, dec, radius):
        if not self.__detectToken(): return
        ssapurl='http://{0}.lamost.org{1}/voservice/ssap?pos={2},{3}&size={4}&token={5}'.format(self.__getDataset(), self.__getVersion(), ra, dec, radius, self.token)
        return self.getUrl(ssapurl)

    def sql(self, sql):
        if not self.__detectToken(): return
        sqlurl='http://{0}.lamost.org{1}/sql/q?&token={2}'.format(self.__getDataset(), self.__getVersion(), self.token)
        return self.getUrl(sqlurl, {'output.fmt':'csv', 'sql':sql})

    def query(self, params):
        if not self.__detectToken(): return
        qurl='http://{0}.lamost.org{1}/q?token={2}'.format(self.__getDataset(), self.__getVersion(), self.token)
        return self.getUrl(qurl, params)
    
    def query2(self, params, files):
        if not self.__detectToken(): return
        qurl='http://{0}.lamost.org{1}/q?token={2}'.format(self.__getDataset(), self.__getVersion(),self.token)
        r=requests.post(qurl, data=params, files=files)
        return str(r.text)
    
    def readFits(self, filename):
        hdulist = pyfits.open(filename)
        head = hdulist[0].header
        scidata = hdulist[0].data
        coeff0 = head['COEFF0']
        coeff1 = head['COEFF1']
        pixel_num = head['NAXIS1'] 
        specflux = scidata[0,]
        spec_noconti = scidata[2,]
        wavelength=numpy.linspace(0,pixel_num,pixel_num)
        wavelength=numpy.power(10,(coeff0+wavelength*coeff1))
        hdulist.close()
        #
        spec_smooth_7=scipy.signal.medfilt(specflux,7)
        spec_smooth_15=scipy.signal.medfilt(specflux,15)
        return (wavelength, specflux, spec_smooth_7, spec_smooth_15)
    

C:\Anaconda3-v5.0.1-x64\lib\site-packages\pyfits-3.5-py3.6-win-amd64.egg\pyfits\__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
wavelength, specflux, spec_smooth_7, spec_smooth_15=lamost().readFits('spec-57278-EG224429N215706B01_sp01-001.fits')
print('wavelength', wavelength)
print('specflux', specflux)
print('spec_smooth_7', spec_smooth_7)
print('spec_smooth_15', spec_smooth_15)

wavelength [3699.98531173 3700.83758098 3701.69004654 ... 9097.03674279 9099.1321902
 9101.22812029]
specflux [153.35196 137.13832 122.05631 ...   0.        0.        0.     ]
spec_smooth_7 [122.05631256 137.13832092 137.13832092 ...   0.           0.
   0.        ]
spec_smooth_15 [73.19073486 73.19073486 73.19073486 ...  0.          0.
  0.        ]


C:\Anaconda3-v5.0.1-x64\lib\site-packages\pyfits-3.5-py3.6-win-amd64.egg\pyfits\util.py:899: UserWarning: Could not find appropriate MS Visual C Runtime library or library is corrupt/misconfigured; cannot determine whether your file object was opened in append mode.  Please consider using a file object opened in write mode instead.
  'Could not find appropriate MS Visual C Runtime '


In [3]:
def showHtml(html):
    from IPython.core.display import display, HTML
    display(HTML(html))

In [4]:
params={'output.fmt':'csv','obsidTextarea':'353301001'}
hl=lamost().query(params)
showHtml(hl)

In [5]:
params={'output.fmt':'csv','pos.type':'proximity'}
files={'pos.posfile':('sample.txt', open('sample.txt', 'r'))}
hl=lamost().query2(params, files)
showHtml(hl)

In [6]:
lamost().sql('select 1')

'#\tPowered by Chinese Virtual Observatory (China-VO): www.china-vo.org 2018-01-29 14:49:39\n#\tIn case of problem: please contact support@lamost.org\n#----------------------------------------------------------------------------\n#\t?column?|null|null|null\n#----------------------------------------------------------------------------\n?column?\n1\n'

In [7]:
lamost().getInfo('353301001')

'{"response":[{"what":"obsid","action":"","data":"353301001"},{"what":"designation","action":"","data":"J224415.49+193359.7"},{"what":"obsdate","action":"","data":"2015-09-12"},{"what":"mjd","action":"","data":"57277"},{"what":"lmjd","action":"","data":"57278"},{"what":"planid","action":"","data":"EG224429N215706B01"},{"what":"spid","action":"","data":"1"},{"what":"fiberid","action":"","data":"1"},{"what":"ra","action":"","data":"341.0645660000"},{"what":"dec","action":"","data":"19.5665910000"},{"what":"class","action":"","data":"STAR"},{"what":"subclass","action":"","data":"K7"},{"what":"z","action":"","data":"0.0000400277"},{"what":"logg","action":"","data":"4.701"},{"what":"teff","action":"","data":"4141.41"},{"what":"feh","action":"","data":"-0.323"},{"what":"rv","action":"","data":"12.00"}]}'

In [8]:
lm=lamost()
#lm.token='FjpGwQlt3d'
lm.dataset=5
lm.version=1

In [9]:
lamost().downloadFits(obsid='353301001',savedir='./')

In [10]:
lamost().downloadPng(obsid='353301007',savedir='./')

In [11]:
lamost().getFitsCsv(obsid='353301007')

'Wavelength,Flux,FluxSmooth,FluxSmooth\n3699.985312,474.216156,474.216156,364.588196\n3700.837581,511.423279,474.216156,364.588196\n3701.690047,493.287415,474.216156,376.633545\n3702.542708,498.277740,474.216156,376.633545\n3703.395567,459.725311,459.725311,388.815247\n3704.248622,432.466461,432.466461,388.815247\n3705.101873,388.815247,388.815247,402.437042\n3705.955321,364.588196,376.633545,409.560608\n3706.808965,358.510040,376.633545,409.560608\n3707.662806,376.633545,376.633545,409.560608\n3708.516844,369.942780,376.633545,409.560608\n3709.371078,402.437042,387.634308,409.560608\n3710.225509,387.634308,402.437042,409.560608\n3711.080137,409.560608,409.560608,409.560608\n3711.934962,442.840942,442.840942,442.840942\n3712.789984,470.741638,468.558655,468.558655\n3713.645203,468.558655,470.741638,468.558655\n3714.500619,529.474854,515.789246,468.558655\n3715.356231,515.789246,515.789246,468.558655\n3716.212041,524.817017,524.817017,468.558655\n3717.068048,538.404724,524.817017,468.55

In [12]:
lamost().getInfo(obsid='353301007')

'{"response":[{"what":"obsid","action":"","data":"353301007"},{"what":"designation","action":"","data":"J224538.96+194751.4"},{"what":"obsdate","action":"","data":"2015-09-12"},{"what":"mjd","action":"","data":"57277"},{"what":"lmjd","action":"","data":"57278"},{"what":"planid","action":"","data":"EG224429N215706B01"},{"what":"spid","action":"","data":"1"},{"what":"fiberid","action":"","data":"7"},{"what":"ra","action":"","data":"341.4123710000"},{"what":"dec","action":"","data":"19.7976180000"},{"what":"class","action":"","data":"STAR"},{"what":"subclass","action":"","data":"G0"},{"what":"z","action":"","data":"-0.0000727837"},{"what":"logg","action":"","data":"3.907"},{"what":"teff","action":"","data":"5842.73"},{"what":"feh","action":"","data":"-0.279"},{"what":"rv","action":"","data":"-21.82"}]}'

In [13]:
lamost().conesearch(ra=10.0004738,dec=40.9952444,radius=0.2)

'<?xml version="1.0" encoding="UTF-8"?>\n<VOTABLE xsi:schemaLocation="http://www.ivoa.net/xml/VOTable/v1.2 http://vo.ari.uni-heidelberg.de/docs/schemata/VOTable-1.2.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.ivoa.net/xml/VOTable/v1.2" version="1.2">\n<RESOURCE type="results">\n<DESCRIPTION>Powered by Chinese Virtual Observatory (China-VO): http://www.china-vo.org Mon Jan 29 14:49:39 CST 2018 In case of problem, please contact:\tsupport@lamost.org\n</DESCRIPTION><INFO name="SERVICE_PROTOCOL" value="1.2">ConeSearch</INFO>\n<INFO name="QUERY_STATUS" value="OK"></INFO>\n<TABLE>\n<FIELD ID="COLID_33326" name="obsid" datatype="char" arraysize="*" ucd="ID_MAIN">\n<DESCRIPTION></DESCRIPTION>\n</FIELD>\n<FIELD ID="COLID_33327" name="designation" datatype="char" arraysize="*" ucd="">\n<DESCRIPTION></DESCRIPTION>\n</FIELD>\n<FIELD ID="COLID_33328" name="obsdate" datatype="char" arraysize="*" ucd="">\n<DESCRIPTION></DESCRIPTION>\n</FIELD>\n<FIELD ID="COLID_33329" 

In [14]:
lamost().ssap(ra=10.0004738,dec=40.9952444,radius=0.2)

'<?xml version="1.0" encoding="UTF-8"?>\n<VOTABLE xmlns:ssa="http://www.ivoa.net/xml/DalSsap/v1.0" xsi:schemaLocation="http://www.ivoa.net/xml/VOTable/v1.2 http://vo.ari.uni-heidelberg.de/docs/schemata/VOTable-1.2.xsd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.ivoa.net/xml/VOTable/v1.2" version="1.2">\n<RESOURCE type="results">\n<DESCRIPTION>Powered by Chinese Virtual Observatory (China-VO): http://www.china-vo.org Mon Jan 29 14:49:39 CST 2018 In case of problem, please contact:\tsupport@lamost.org\n</DESCRIPTION><INFO name="SERVICE_PROTOCOL" value="1.1">SSAP</INFO>\n<INFO value="OK" name="QUERY_STATUS"></INFO>\n<TABLE name="result">\n<PARAM name="DataModel" utype="ssa:Dataset.DataModel" datatype="char" arraysize="*" value="Spectrum 1.0" />\n<PARAM name="Publisher" utype="ssa:Curation.Publisher" ucd="meta.curation" datatype="char" arraysize="*" value="Chinese Virtual Observatory" />\n<FIELD ID="COLID_33326" name="obsid" utype="" datatype="long" ucd="meta.i